<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfPBCGLDxsK-f0Wnhf9jOj4HnYDCAA8QFvPw&s" width="200">

# 01MIAR - Actividad Whitepapers

# 1. Introducción


## 1.1. Contexto

El análisis de datos es una disciplina con una importancia creciente, ya que supone una herramienta muy potente para orientar la toma de decisiones o
para la automatización de procesos.
Un tipo de dato muy relevante en el análisis de datos es el texto, debido a que cada vez hay más datos de este tipo.
Sin embargo, para analizarlo normalmente hacen falta técnicas especiales, por lo que se le tiene a prestar menor atención, incluso en ámbitos académicos.
Estas particularidades, sumadas a un problema común en aprendizaje supervisado, como es la escasez de individuos etiquetados, dificulta las tareas supervisadas de análisis de texto.


## 1.2. Motivación

El artículo elegido para esta actividad, *Small-Text: Active Learning for Text Classification in Python* de Schröder C. et al (2021) [1], está relacionado con esta problemática, y presenta una librería que implementa una metodología de aprendizaje activo (*active learning*) para clasificación de texto.
El aprendizaje activo es una técnica que pretende minimizar el coste de entrenar un modelo supervisado, mientras que intenta optimizar su rendimiento.
Esto lo hace utilizando el menor número posible de datos etiquetados que permitan obtener un rendimiento razonable.
Este tipo de técnicas no se habían implementado especialmente orientadas para la clasificación de texto en Python, debido a las características especiales del texto.
Los autores de este artículo presentan una librería que proporciona esta funcionalidad, permitiendo abaratar por tanto el entrenamiento de modelos de clasificación de texto.


## 1.3. Estructura del cuaderno

Este cuaderno sigue la estructura del propio artículo, resumiendo los contenidos de cada sección y reflexionando sobre los mismos.
La Sección 2 introduce la librería de `small_text` y la metodología de aprendizaje activo.
La Sección 3 presenta una aclaración sobre la fase de etiquetado de los datos por expertos, y la Sección 4 expone un ejemplo de uso de la librería `small_text` utilizando el código propuesto por los autores. Por último, la Sección 5 expone las conclusiones del análisis crítico del artículo.

# 2. Librería `small-text`

La libería `small-text` está diseñada para facilitar el desarrollo de un modelo de clasificación de texto, proporcionando desde un único lugar todos los elementos necesarios.
Para entender los elementos de esta librería y sus funciones, es necesario tener alguna noción sobre el proceso de aprendizaje activo.
Los principales elementos en una metodología de aprendizaje activo son un clasificador, una estrategia de selección y un criterio de parada. En la siguiente imagen, extraída de la web [cloudfactory.com](https://www.cloudfactory.com), se muestran los pasos del aprendizaje activo.

<div style="text-align: center;">
  <img
    src="https://www.cloudfactory.com/hs-fs/hubfs/04-blog-img/Scalable-Active-Learning-for-Autonomous-Driving.png?width=2130&name=Scalable-Active-Learning-for-Autonomous-Driving.png" width="700"
    alt="Elementos de un proceso de aprendizaje activo"
  >
</div>

1. En primer lugar, se entrena un modelo con un pequeño conjunto de datos etiquetados.
2. Después, se utiliza una estrategia de selección (*query strategy*) para determinar qué individuos son más informativos y deben ser etiquetados.
3. A continuación, uno o varios expertos etiquetan los individuos seleccionados.
4. Por último, se reentrena el modelo con los nuevos datos, y se repite el proceso hasta que se cumpla un determinado criterio de parada.


Contenido de la librería:

* **Clasificadores**: Está integrada con `scikit-learn`, permitiendo usar todos sus clasificadores.
* **Estrategias de selección**: Implementa 13 estrategias de selección diferentes, cubriendo 4 paradigmas distintos: estrategias basadas en confianza, basadas en embeddings, basadas en gradiente y las basadas en 'coresets'. Entre estas estrategias se incluyen las habituales además de algunas especiales para texto.
* **Criterios de parada**: Incluye 5 criterios diferentes: predicciones estables, incertidumbre global, cambio en clasificación, cambio en F-measure y número límite de iteraciones.
* **Integración con `transformers`**: Permite usar los modelos de lenguaje más modernos, lo que permite aprovechar el uso de GPUs en caso de estar disponibles.


En definitiva, esta librería proporciona todos los elementos necesarios la clasificación de texto a escala industrial. Además, ofrece una interfaz sencilla y cómoda que se integra con las librerías más significativas de aprendizaje automático como `scikit-learn`, `torch` y `transformers`.

# 3. Etiquetado por expertos

Los autores establecen una diferencia clara entre las herramientas o software de etiquetado y las librerías.
Desde su punto de vista, su naturaleza es completamente distinta, dado que las herramientas de etiquetado están pensadas para interaccionar con usuarios y normalmente tienen una interfaz gráfica.
Por este motivo, los autores han preferido diseñar la librería como un 'standalone', para facilitar su uso en cualquier tipo de entorno.
De esta manera, no condicionan el uso de una interfaz específica, y se pueden centrar en mantener un diseño agnóstico que sea fácilmente integrable en cualquier tipo de organización o proceso.
En este sentido, los autores destacan que su librería se podría utilizar como un 'backend' de una aplicación web, con herramientas de etiquetado existentes o en un contexto experimental.

# 4. Ejemplo de uso

Los autores también proponen un ejemplo simple de uso de su librería implementando un clasificador de spam.
En él, demuestran como se puede utilizar la librería en un proceso de aprendizaje activo.
Esta sección reproduce este mismo ejemplo, aunque usando un modelo más simple al no tener disponible una GPU.
Los pasos del ejemplo son los siguientes:

1. Instalación de la librería
2. Crear los datos de ejemplo
3. Configurar los parámetros del aprendizaje activo
4. Inicializar el modelo
5. Comenzar el bucle de aprendizaje activo


## 4.1. Instalación

La librería se puede instalar desde pip, por lo que se puede usar el siguiente comando:

```bash
pip install small_text
```


## 4.2. Datos de ejemplo

Para el ejemplo, se crea un pequeño conjunto de datos que consta de 100 pequeños textos y 2 categorías. Cada categoría contiene el mismo texto, que son: 'this is ham' y 'this is spam'.
Los autores en su ejemplo proponen el uso del modelo 'bert-base-uncased', que ha resultado ser bastante costoso computacionalmente. Al no disponer de una GPU, se ha intercambiado este modelo por 'albert-base-v2', que es más simple.

In [22]:
import numpy as np
from small_text import TransformersDataset, TransformerModelArguments
from transformers import AutoTokenizer


# Fake data example:
# 50 spam and 50 non-spam examples
text = np.array(['this is ham'] * 50 + ['this is spam'] * 50)
labels = np.array([0] * 50 + [1] * 50)

transformer_model = 'albert-base-v2'
tokenizer = AutoTokenizer.from_pretrained(transformer_model)

train = TransformersDataset.from_arrays(
    text, labels, tokenizer,
    target_labels = np.array([0, 1]),
    max_length = 10
)

/usr/local/lib/python3.11/dist-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


## 4.3. Configuración del aprendizaje activo

En este paso se definen las estrategias de selección y el modelo de clasificación.

In [23]:
from small_text import LeastConfidence, TransformerBasedClassificationFactory as TransformerFactory


# Modelo de clasificación
num_classes = 2
model_args = TransformerModelArguments(transformer_model)
clf_factory = TransformerFactory(model_args, num_classes)

# Estrategia de selección
query_strategy = LeastConfidence()

## 4.4. Inicialización

El paso de inicialización entrena un modelo con un conjunto pequeño de datos, a partir del cual se comenzará con el aprendizaje activo.
En este caso se entrena el modelo utilizando una muestra de 10 textos, ya que en este caso se conocen las etiquetas y no es necesaria interacción humana.
En escenarios alternativos se podría usar una estrategia de 'cold start', o empezar directamente con alguna estrategia de selección y etiquetar una pequeña muestra.

In [24]:
from small_text import PoolBasedActiveLearner, random_initialization_balanced as init


active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)

# Provide initial data.
indices_initial = init(train.y, n_samples = 10)
active_learner.initialize_data(
    indices_initial,
    train.y[indices_initial]
)

# 4.5. Bucle de aprendizaje activo

En este paso se realiza el aprendizaje activo, que es el elemento central de la metodología.
En este bucle se realizan un máximo de 5 iteraciones, aunque como indican los autores, se podría utilizar otro criterio de parada con muy pocas líneas de código adicionales.
Para simular el etiquetado de los textos que se seleccionan, se toman sus etiquetas del dataset creado al principio.
Despues, el modelo se actualiza con la función `update`, y se imprime por pantalla la exactitud (*accuracy*) del modelo.

In [25]:
from sklearn.metrics import accuracy_score


num_queries = 5
for i in range(num_queries):
    # Query 10 samples per iteration.
    indices_queried = active_learner.query(num_samples = 10)

    # Simulate user interaction here.
    # Replace this for real-world usage.
    y = train.y[indices_queried]

    # Provide labels for the queried indices.
    active_learner.update(y)

    # Evaluate accuracy on the train set
    print(f'Iteration {i + 1}')
    y_pred = active_learner.classifier.predict(train)
    print('Train accuracy: {:.2f}'.format(accuracy_score(y_pred, train.y)))

Iteration 1
Train accuracy: 1.00
Iteration 2
Train accuracy: 1.00
Iteration 3
Train accuracy: 1.00
Iteration 4
Train accuracy: 1.00
Iteration 5
Train accuracy: 1.00


# 5. *SetFit* con aprendizaje activo

La técnica de *SetFit* es una técnica novedosa de *fine-tuning* que fue presentada por investigadores de Huggingface en 2022, y que presenta muchas mejoras con respecto al *fine-tuning* convencional.
Esta técnica resulta especialmente eficaz cuando se disponen de pocos ejemplos etiquetados, que es el mismo problema al que se enfrenta el aprendizaje activo.
Exponer esta técnica no es el objetivo del estudio, aunque los autores la utilizan para mostrar la eficacia y el interés del aprendizaje activo con su librería.
En el artículo, los autores ajustan un modelo de clasificación de texto con *SetFit* y lo comparan a un modelo con un ajuste convencional.
En sus resultados encuentran que la técnica *SetFit* mejora la exactitud del modelo, y que además, la métrica AUC (*area under the curve*) presenta una mejora bastante significativa.

# 6. Conclusiones

La clasificación de texto es una tarea muy relevante pero que presenta una gran cantidad de retos.
En este cuaderno se han podido exponer dos de ellos, como son la poca disponibilidad de datos etiquetados y el alto coste computacional de entrenar los modelos.
Para poder superar estos retos, la metodología de aprendizaje activo es una gran aliada, ya que con ella podemos abaratar significativamente el entrenamiento de cualquier modelo de clasificación.
Esta reducción de costes es muy importante en el caso del texto, ya que los modelos más modernos tienen un coste base muy elevado.
Manejando estas escalas de gasto, cualquier reducción del coste supone un ahorro de recursos muy significativo, en términos energéticos, monetarios y temporales.
Además, los beneficios de esta metodología se pueden incrementar de la mano de otras técnicas como *SetFit*, que aplican los autores en el artículo.
Por último, también cabe destacar la librería `small_text`, que permite aprovechar todos estos beneficios desde Python, utilizando una interfaz sencilla y familiar.

# Referencias

[1] Christopher Schröder, Lydia Müller, Andreas Niekler, and Martin Potthast (2023). Small-Text: Active Learning for Text Classification in Python. *arXiv preprint arXiv:2107.10314*. [https://arxiv.org/abs/2107.10314](https://arxiv.org/abs/2107.10314).